<a href="https://colab.research.google.com/github/yunnsss/Product-Recommendation-System-for--OliveYoung-Moisturizers/blob/main/%EC%98%AC%EB%A6%AC%EB%B8%8C%EC%98%81_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#세팅

In [ ]:
!pip install selenium
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,556 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu j

In [ ]:
!python --version

import selenium
print(selenium.__version__)

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import numpy as np
import chromedriver_autoinstaller  # setup chrome options

chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

chromedriver_autoinstaller.install()  # set the target URL

Python 3.10.12
4.31.1


#리뷰정보 수집(각각따로 데이터프레임 변환후 병합)

userID

In [ ]:
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# ChromeDriver 설정
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('lang=ko_KR')
chrome_options.add_argument(
    'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
)

def collect_user_ids(product_url):
    user_ids = []
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(product_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="reviewInfo"]/a')))
    review_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="reviewInfo"]/a')))
    review_button.click()
    time.sleep(5)  # 페이지가 로드될 시간을 충분히 제공

    for current_page in range(1, 6):
        try:
            review_section = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'review_cont')))
            driver.execute_script("arguments[0].scrollIntoView(true);", review_section)
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.review_cont')

            for j in range(1, len(review_elements) + 1):
                try:
                    # j 값을 사용하여 각 리뷰의 유저 ID 요소 가져오기
                    user_ID_element = driver.find_element(By.XPATH, f'/html/body/div[3]/div[8]/div/div[9]/div/div[7]/ul/li[{j}]/div[1]/div/p[1]/a[1]')
                    onclick_value = user_ID_element.get_attribute("onclick")

                    # 정규 표현식으로 't_profile_name' 값 추출
                    match = re.search(r"t_profile_name:\s*'([^']*)'", onclick_value)
                    if match:
                        user_ID = match.group(1)
                    else:
                        user_ID = "N/A"

                    user_ids.append(user_ID)
                except NoSuchElementException:
                    user_ids.append("N/A")
                    continue

            if current_page < 5:
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'next'))
                )
                next_page_button.click()
                time.sleep(5)
        except Exception as e:
            print(f"페이지 {current_page} 처리 중 오류 발생: {e}")
            break

    driver.quit()
    user_ids_df = pd.DataFrame(user_ids, columns=['유저 ID'])
    return user_ids_df

# 유저 ID 수집 실행
product_url = "https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000164634&t_page=%ED%86%B5%ED%95%A9%EA%B2%80%EC%83%89%EA%B2%B0%EA%B3%BC%ED%8E%98%EC%9D%B4%EC%A7%80&t_click=%EA%B2%80%EC%83%89%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_search_name=%EC%88%98%EB%B6%84%ED%81%AC%EB%A6%BC&t_number=15&dispCatNo=1000001000700070013&trackingCd=Result_11_30"
user_ids_df = collect_user_ids(product_url)
print(user_ids_df)



               유저 ID
0        kimzhenghai
1                 링규
2        kimzhenghai
3                 링규
4           kami****
5            Shong15
6                꽁언니
7               곰네마리
8             im****
9                정예원
10  gfdsfgdfasdfasfg
11        aihyos****
12          cjcj****
13         qlsdl****
14         yhs06****
15         hangj****
16           lov****
17          suim****
18        mminge****
19        alstjr****
20              인천소년
21                키진
22              구이92
23              인천소년
24             올영겟하자
25          kangin피부
26              파오ㄹI
27             포에버올영
28              그올블올
29               근근근
30              슈크림츄
31              슈크림츄
32              yyjj
33             냠냐밍도리
34               애니7
35              슈크림츄
36              슈크림츄
37              yyjj
38             Tleun
39                피비
40             Jjj09
41             Jjj09
42               김펭군
43              달이님임
44              찡징찡징
45              찡징찡징
46           

기타제품정보

In [ ]:
def collect_review_details(product_url):
    review_details = []
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(product_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="reviewInfo"]/a')))
    review_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="reviewInfo"]/a')))
    review_button.click()
    time.sleep(5)



    reviews_data = []

    # 드라이버 초기화
    driver = webdriver.Chrome(options=chrome_options)
    # 상품 페이지로 이동
    driver.get(product_url)

    # 페이지 로딩 대기
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="reviewInfo"]/a')))

    # 상품명 크롤링 (PK)
    product_name_element = driver.find_element(By.CSS_SELECTOR, "p.prd_name")
    product_name = product_name_element.text.strip()

    # 브랜드명 크롤링
    brand_name_element = driver.find_element(By.CSS_SELECTOR, "p.prd_brand")
    brand_name = brand_name_element.text.strip()

    # '리뷰' 버튼을 클릭하기 위해 WebDriverWait 사용
    review_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="reviewInfo"]/a'))
    )
    review_button.click()

    for current_page in range(1, 6):
        try:
            review_section = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'review_cont')))
            driver.execute_script("arguments[0].scrollIntoView(true);", review_section)
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.review_cont')
            for review in review_elements:
                try:
                    skin_type_text = review.find_element(By.XPATH, './/dl[@class="poll_type1"][dt/span[text()="피부타입"]]/dd').text.strip()
                    skin_concern_text = review.find_element(By.XPATH, './/dl[@class="poll_type1"][dt/span[text()="피부고민"]]/dd').text.strip()
                    irritation_level_text = review.find_element(By.XPATH, './/dl[@class="poll_type1"][dt/span[text()="자극도"]]/dd').text.strip()
                    review_point_text = review.find_element(By.XPATH, './/div[@class="score_area"]//span[@class="point"]').text.strip()
                    review_content = review.find_element(By.CSS_SELECTOR, 'div.txt_inner').text.strip()
                    review_details.append({
                        '브랜드': brand_name,
                        '상품명': product_name,
                        '피부타입': skin_type_text,
                        '피부고민': skin_concern_text,
                        '자극도': irritation_level_text,
                        '별점': review_point_text,
                        '한달이상사용': 1 if '한달' in review_content else 0,
                        '재구매여부': 1 if '재구매' in review_content else 0
                    })
                except Exception:
                    continue
            if current_page < 5:
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'next'))
                )
                next_page_button.click()
                time.sleep(5)
        except Exception as e:
            print(f"페이지 {current_page} 처리 중 오류 발생: {e}")
            break

    driver.quit()
    review_details_df = pd.DataFrame(review_details)
    return review_details_df

# 리뷰 세부사항 수집 실행
review_details_df = collect_review_details(product_url)
print(review_details_df)

    브랜드                                       상품명      피부타입     피부고민  \
0   플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)  복합성에 좋아요  보습에 좋아요   
1   플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)  복합성에 좋아요  보습에 좋아요   
2   플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)   지성에 좋아요  보습에 좋아요   
3   플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)   건성에 좋아요  보습에 좋아요   
4   플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)   건성에 좋아요  보습에 좋아요   
5   플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)   건성에 좋아요  진정에 좋아요   
6   플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)   건성에 좋아요  보습에 좋아요   
7   플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)   지성에 좋아요  보습에 좋아요   
8   플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)   지성에 좋아요  진정에 좋아요   
9   플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)   지성에 좋아요  보습에 좋아요   
10  플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)  복합성에 좋아요  보습에 좋아요   
11  플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)  복합성에 좋아요  진정에 좋아요   
12  플리프  플리프 시카-알로에 프레쉬 수분크림 150ml 기획세트(+50ml 증정)   건성에 좋아요  보습에

skin 정보

In [ ]:
def collect_skin_info(product_url):
    skin_info = []
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(product_url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="reviewInfo"]/a')))
    review_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="reviewInfo"]/a')))
    review_button.click()
    time.sleep(5)

    for current_page in range(1, 6):
        try:
            review_section = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'review_cont')))
            driver.execute_script("arguments[0].scrollIntoView(true);", review_section)
            review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.review_cont')
            for j in range(1, len(review_elements) + 1):
                try:
                    skin_text = driver.find_element(By.XPATH, f'/html/body/div[3]/div[8]/div/div[9]/div/div[7]/ul/li[{j}]/div[1]/div/p[2]').text.strip()
                    skin_info.append(skin_text)
                except Exception:
                    skin_info.append("N/A")
                    continue
            if current_page < 5:
                next_page_button = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.CLASS_NAME, 'next'))
                )
                next_page_button.click()
                time.sleep(5)
        except Exception as e:
            print(f"페이지 {current_page} 처리 중 오류 발생: {e}")
            break

    driver.quit()
    skin_info_df = pd.DataFrame(skin_info, columns=['피부'])
    return skin_info_df

# 피부 정보 수집 실행
skin_info_df = collect_skin_info(product_url)
print(skin_info_df)

                   피부
0                 N/A
1      지성 쿨톤 블랙헤드 트러블
2                 N/A
3      지성 쿨톤 블랙헤드 트러블
4                 N/A
5                 N/A
6        민감성 쿨톤 각질 모공
7                 N/A
8           트러블성 가을웜톤
9             민감성 민감성
10                N/A
11                N/A
12                N/A
13                N/A
14                N/A
15                N/A
16                N/A
17                N/A
18                N/A
19                N/A
20       복합성 웜톤 모공 미백
21  트러블성 겨울쿨톤 민감성 트러블
22                N/A
23       복합성 웜톤 모공 미백
24      건성 겨울쿨톤 각질 모공
25        지성 웜톤 모공 미백
26      건성 가을웜톤 모공 탄력
27     복합성 가을웜톤 모공 미백
28      건성 여름쿨톤 각질 모공
29                N/A
30             복합성 웜톤
31             복합성 웜톤
32                N/A
33                N/A
34     건성 쿨톤 민감성 블랙헤드
35             복합성 웜톤
36             복합성 웜톤
37                N/A
38                N/A
39           건성 쿨톤 주름
40                N/A
41                N/A
42                N/A
43                N/A
44        

#데이터병합

In [ ]:
def merge_data(user_ids_df, review_details_df, skin_info_df):
    merged_df = pd.concat([user_ids_df, review_details_df, skin_info_df], axis=1)
    return merged_df

# 데이터 병합 실행
reviews_df = merge_data(user_ids_df, review_details_df, skin_info_df)
display(reviews_df)

,유저 ID,브랜드,상품명,피부타입,피부고민,자극도,별점,한달이상사용,재구매여부,피부
0,옥님,에스네이처,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,건성에 좋아요,보습에 좋아요,자극없이 순해요,5점만점에 5점,0,0,중성 쿨톤 미백 잡티
1,채르,에스네이처,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,복합성에 좋아요,보습에 좋아요,자극없이 순해요,5점만점에 5점,0,0,건성 웜톤 잡티
2,플로우플로우,에스네이처,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,복합성에 좋아요,보습에 좋아요,자극없이 순해요,5점만점에 5점,0,0,복합성 웜톤 다크서클 모공
3,내게올령,에스네이처,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,건성에 좋아요,보습에 좋아요,자극없이 순해요,5점만점에 5점,0,0,건성 웜톤 주름 탄력
4,SOLE,에스네이처,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,건성에 좋아요,보습에 좋아요,자극없이 순해요,5점만점에 5점,0,0,건성 웜톤 미백 잡티
5,다산정약용,에스네이처,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,건성에 좋아요,보습에 좋아요,자극없이 순해요,5점만점에 5점,0,0,건성 여름쿨톤 모공 블랙헤드
6,말티츄,에스네이처,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,건성에 좋아요,보습에 좋아요,자극없이 순해요,5점만점에 3점,0,0,지성 여름쿨톤 모공 잡티
7,하쿠나마타타고래꼬리,에스네이처,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,건성에 좋아요,보습에 좋아요,자극없이 순해요,5점만점에 5점,0,0,트러블성 봄웜톤 모공 민감성
8,유진입니다,에스네이처,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,건성에 좋아요,진정에 좋아요,자극없이 순해요,5점만점에 5점,0,0,약건성 가을웜톤 탄력
9,올챙이포뇨,에스네이처,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,건성에 좋아요,보습에 좋아요,자극없이 순해요,5점만점에 4점,0,1,건성 봄웜톤 미백 민감성


#csv파일로 업로드

In [ ]:
# 데이터프레임을 CSV 파일로 저장
reviews_df.to_csv('user리뷰_df.csv', index=True, encoding='utf-8-sig')